In [1]:
from sqlalchemy import create_engine
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import descartes
import psycopg2  
import subprocess 
import sys, os
import math

import stac
from osgeo import gdal, ogr, osr
import rasterio

from datetime import datetime

import ipywidgets as widgets

## Aquisição dos parâmetros de seleção do usuário

In [2]:
# Inserir "ambos"

sensor = widgets.ToggleButtons(
    options=[('Landsat 8','LC8SR-1'),('Sentinel 2','S2_10-1')],
    description='Selecione o sensor:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
#     tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
#     icons=['check'] * 3
)

sensor

ToggleButtons(description='Selecione o sensor:', options=(('Landsat 8', 'LC8SR-1'), ('Sentinel 2', 'S2_10-1'))…

## Obtenção dos itens da coleção do BDC

In [ ]:
# Conectar o serviço da stac
bdc_stac_service = stac.STAC('http://brazildatacube.dpi.inpe.br/')

# Obter itens filtrados pelo objeto JSON (https://stacspec.org/STAC-api.html#operation/postSearchSTAC)
item = bdc_stac_service.search({'collections':[sensor.value], 
                                'limit': 50000})

# Exibir as quantidades de itens filtrados
print("Filtragem Final -> localização e data: ", len(item.features))

## Conexão com o banco de dados 'bdc3'

In [ ]:
conn = psycopg2.connect(database="bdc3", user="postgres", host="localhost", password="postgres") 
cursor = conn.cursor()

### Criar tabela com os metadados das imagens a serem inseridas

Essa tabela já pode ser inserida no momento da criação do banco. Nesse caso, a célula seguinte seria desconsiderada.

**<center>Colunas da tabela</center>**

| id        | collection           | date_img  | cover |
| :-------------:|:-------------:|:-----:|:-----:|
| VARCHAR(60) NOT NULL UNIQUE     | VARCHAR(30) | DATE | REAL |

In [ ]:
# Criar tabela
sql = "CREATE TABLE metadata_metrics(id VARCHAR(60) NOT NULL UNIQUE,pathrow CHAR(6), collection VARCHAR(30), date_img DATE, cover REAL);"
cursor.execute(sql)
conn.commit()

### Importação das imagens filtradas para o BD (cmd)

In [ ]:
# Criação das variáveis do sistema para utilização do CMD
os.environ['PATH'] = r';C:\Program Files\PostgreSQL\11\bin'
os.environ['PGHOST'] = 'localhost'
os.environ['PGPORT'] = '5432'
os.environ['PGUSER'] = 'postgres'
os.environ['PGPASSWORD'] = 'postgres'
os.environ['PGDATABASE'] = 'bdc3'

# iteração em cada item adquirido para inserção das imagens no DB "bdc3"
for i in item.features:
    
    # Adquirir os dados das imagens
    name = i['id']
    date = i['properties']['datetime']
    collection = i['collection']
    cloud = str(i['properties']['eo:cloud_cover'])
    
    if collection == 'LC8SR-1':
        path_row = i['id'][10:16]
    elif collection == 'S2_10-1':
        path_row = i['id'][11:17]
    
    if cloud == 'None':
        print(name)
    else:
        # Inserir a referencia do input na tabela de metadados
        sql = "INSERT INTO metadata_metrics (id, collection, pathrow, date_img, cover) VALUES ('" + name + "','" + collection + "','" + path_row + "','" + date + "'::date, "+ cloud + ");"
        cursor.execute(sql)
        conn.commit()

### Apagar tabela (Interno)

In [ ]:
sql = "DROP TABLE IF EXISTS metadata_metrics"
cursor.execute(sql)
conn.commit()